In [3]:
import os
import re

In [4]:
root_dir = os.path.abspath('../')
data_dir = os.path.join(root_dir, 'data')
res_dir = os.path.join(root_dir, 'res')
res_filtered_dir = os.path.join(root_dir, 'filtered')
res_raw_dir = os.path.join(root_dir, 'raw')
res_raw_vocab = os.path.join(res_raw_dir, 'vocab')
res_raw_wti = os.path.join(res_raw_dir, 'wti')
res_raw_embeddings = os.path.join(res_dir, 'embed')
res_filt_vocab = os.path.join(res_filtered_dir, 'vocab')
res_filt_wti = os.path.join(res_filtered_dir, 'wti')
res_filt_embeddings = os.path.join(res_filtered_dir, 'embed')
raw_data_dir = os.path.join(data_dir,'raw_data')
clean_data_dir = os.path.join(data_dir, 'clean_data')
embeddings_dir = os.path.join(data_dir, 'glove.twitter.27B')
embeddings_file = os.path.join(embeddings_dir, 'glove.twitter.27B.200d.txt')
print embeddings_file

/home/vasanth/programming/690N/project/conversational_agent_personified/data/glove.twitter.27B/glove.twitter.27B.200d.txt


In [5]:
def construct_vocabulary():
    vocabulary = {}
    vocab_list = []
    embeddings = []
    print "Start filtering word embeddings..."
    for sub in os.listdir(clean_data_dir):
        if sub.startswith("."): 
            continue  # ignore other files like .DS_Store
        else:
            sub_dir = os.path.join(clean_data_dir,sub)
            for file_name in os.listdir(sub_dir):
                if file_name.startswith("."):
                    continue  # ignore other files like .DS_Store
                else:
                    full_filename = os.path.join(sub_dir,file_name)
                    with open(full_filename) as f1:
                        lines = f1.readlines()
                        for i, line in enumerate(lines):
                            words = line.split(" ")
                    #if vocabulary.has_key()

In [6]:
def question_answer_format():
    vocabulary = {}
    if not os.path.exists(res_dir):
        os.makedirs(res_dir)
    questions_file = os.path.join(res_dir,'input1.txt')
    answers_file = os.path.join(res_dir,'input2.txt')
    questions_file1 = os.path.join(res_dir,'input3.txt')
    answers_file1 = os.path.join(res_dir,'input4.txt')    
    print "Start writing ques answer files..."
    import glob
    lines_written = 0
    for full_filename in glob.iglob(clean_data_dir + '/*/*.txt'):
        with open(full_filename) as f1:
            lines = f1.readlines()
            with open(questions_file, 'a') as f2, open(answers_file, 'a') as f3, open(questions_file1, 'a') as f4, open(answers_file1, 'a') as f5:                            

                for i, line in enumerate(lines):
                        line = line.split(":")[1]
                        line1 = re.sub(r'\([^)]*\)', '', line).strip(" ")
                        f2.write(line)
                        f4.write(line1)
                        if lines_written == 0:
                            pass          
                        elif lines_written % 100000 == 0:
                            f3.write(line)
                            f5.write(line1)
                            print "Lines written: {0}".format(lines_written)
                            print "Writing from {0}".format(full_filename.split("/")[-1])
                        else:
                            f3.write(line)
                            f5.write(line1)
                        lines_written += 1
    print "Finished writing all files"

In [15]:
question_answer_format()

Start writing ques answer files...
Finished writing all files


In [7]:
def loadWordEmbeddings(filename):
    vocabulary = []
    word_to_index = {}
    embeddings = []
    i = 0
    file_point = 0
    with open(filename,'r') as f:
        for line in f.readlines():
            one_record = line.strip().split(" ")
            vocabulary.append(one_record[0])
            word_to_index[one_record[0]] = len(vocabulary)-1
            embeddings.append(one_record[1:])
            i += 1
            if i%5000 == 0:
                print "Embedding of {0} words done".format(str((i+1)*50))
            if i%50000000 == 0 and i > 0:
                print "Now pickling word_to_index dict..."
                wti = "wti"+str(file_point)
                embed = "embedding"+str(file_point)
                wti_filename = os.path.join(res_raw_wti, wti)
                embed_filename = os.path.join(res_raw_embeddings, embed)
                with open(wti_filename, 'wb') as handle:
                    pickle.dump(word_to_index, handle, protocol=pickle.HIGHEST_PROTOCOL)
                print "Pickling word_to_index dict complete."
                print "Now pickling embeddings array..."
                np.save(embed_filename, embeddings, allow_pickle=True)
                print "Pickling embeddings array complete."                
                print "Loaded embeddings!!"
                vocabulary = []
                word_to_index = {}
                embeddings = []
            return vocabulary, word_to_index, embeddings

In [8]:
embd_file_dir = os.path.join(data_dir, 'glove.twitter.27B')
embd_file_name = os.path.join(embd_file_dir,'glove.twitter.27B.200d.txt')
loadWordEmbeddings(embd_file_name)

(['<user>'],
 {'<user>': 0},
 [['0.31553',
   '0.53765',
   '0.10177',
   '0.032553',
   '0.003798',
   '0.015364',
   '-0.20344',
   '0.33294',
   '-0.20886',
   '0.10061',
   '0.30976',
   '0.50015',
   '0.32018',
   '0.13537',
   '0.0087039',
   '0.1911',
   '0.24668',
   '-0.060752',
   '-0.43623',
   '0.019302',
   '0.59972',
   '0.13444',
   '0.012801',
   '-0.54052',
   '0.27387',
   '-1.182',
   '-0.27677',
   '0.11279',
   '0.46596',
   '-0.090685',
   '0.24253',
   '0.15654',
   '-0.23618',
   '0.57694',
   '0.17563',
   '-0.01969',
   '0.018295',
   '0.37569',
   '-0.41984',
   '0.22613',
   '-0.20438',
   '-0.076249',
   '0.40356',
   '0.61582',
   '-0.10064',
   '0.23318',
   '0.22808',
   '0.34576',
   '-0.14627',
   '-0.1988',
   '0.033232',
   '-0.84885',
   '-0.25684',
   '0.26369',
   '0.29562',
   '0.1847',
   '-0.20668',
   '-0.013297',
   '0.12233',
   '-0.47751',
   '-0.17202',
   '-0.14577',
   '0.047446',
   '-0.15824',
   '0.054215',
   '-0.19426',
   '-0.08148

In [11]:
construct_vocabulary()

Start filtering word embeddings...
